In [1]:
import ecomis
import pandas as pd
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
import numpy as np
from datetime import datetime
import time
import posixpath
import logging

logger = logging.getLogger("juplogger")
handler = ecomis.LogViewver()
logger.addHandler(handler)
logger.setLevel(logging.INFO)
logger.propagate = False

month = '201912'

inifile = ecomis.ReadIniFile(mercado="empresas")
defaultpath = inifile.getDataPath()
testpath = inifile.getTestPath()

handler.show_logs()

# Configurando los archivos de configuración para administrar la base de datos
dbparser = inifile.getDbIniFileParser()
dbmanager = ecomis.DbDataProcess(month)
dbmanager.setParser(dbparser)

Output(layout=Layout(border='solid', height='250px', width='100%'), outputs=({'name': 'stdout', 'output_type':…

In [2]:
cb1 = widgets.Checkbox(
    value=False,
    description='Cuotas',
    disabled=False
)

cb2 = widgets.Checkbox(
    value=False,
    description='Resultados',
    disabled=False
)

w = widgets.RadioButtons(
    options=['Corporaciones', 'Grandes Clientes, DDNN, IS', 'Venta Regional Empresas', 'Pymes y Dealers'],
    description='Canal:',
    disabled=False
)
display(cb1)
display(cb2)
display(w)

Checkbox(value=False, description='Cuotas')

Checkbox(value=False, description='Resultados')

RadioButtons(description='Canal:', options=('Corporaciones', 'Grandes Clientes, DDNN, IS', 'Venta Regional Emp…

In [3]:
cb2.value

False

In [4]:
#Carga de Panel de voz
#*************************
# Agregar mes enero faltante en cuotas de GC
# Asegúrate en colocar 0 en la cabecera de los resultados y eliminar toda la data fuera del formato
#"""
section_logins = ecomis.SectionObj(inifile,'Logins',month)
logins = ecomis.LoadFileProcess(section_logins).loadFile()

periodo = section_logins.getParameter('periodo')

section_metricasconjuntas = ecomis.SectionObj(inifile,'Metricas_conjuntas',month)
metricasconjuntas = ecomis.LoadFileProcess(section_metricasconjuntas).loadFile()

if w.value == 'Corporaciones':
    section_kpisvoz = ecomis.SectionObj(inifile,'Kpis_voz_corpo',month)
    section_comisionantesvoz = ecomis.SectionObj(inifile,'Comisionantes_voz_corpo',month)
elif w.value == 'Grandes Clientes, DDNN, IS':
    section_kpisvoz = ecomis.SectionObj(inifile,'Kpis_voz_gc_dd_is',month)
    section_comisionantesvoz = ecomis.SectionObj(inifile,'Comisionantes_voz_gc_dd_is',month)
elif w.value == 'Venta Regional Empresas':
    section_kpisvoz = ecomis.SectionObj(inifile,'Kpis_voz_venta_regional',month)
    section_comisionantesvoz = ecomis.SectionObj(inifile,'Comisionantes_venta_regional',month)    
elif w.value == 'Pymes y Dealers':
    section_kpisvoz = ecomis.SectionObj(inifile,'Kpis_voz_pymes',month)
    section_comisionantesvoz = ecomis.SectionObj(inifile,'Comisionantes_voz_pymes',month)  

if cb1.value:
    if w.value == 'Venta Regional Empresas':
      section_cuotasvoz = ecomis.SectionObj(inifile,'Cuotas_Regiones_Pymes',month)
    else:
      section_cuotasvoz = ecomis.SectionObj(inifile,'Cuotas',month)
else:
    cuotasvoz = pd.DataFrame()

if cb2.value:
    # Preparar los resultados antes de enviar el cerrado
    # GCE_Resultados debe tener dos o mas Hojas sino lanza error
    section_resultadosvoz = ecomis.SectionObj(inifile,'Resultados',month)
else:
    resultadosvoz = pd.DataFrame()

kpisvoz = ecomis.LoadFileProcess(section_kpisvoz).loadFile()
comisionantesvoz = ecomis.LoadFileProcess(section_comisionantesvoz).loadFile()

if cb1.value:
  cuotasvoz = ecomis.LoadFileProcess(section_cuotasvoz).loadFile()

if cb2.value:
  resultadosvoz = ecomis.LoadFileProcess(section_resultadosvoz).loadFile()


#resultadosvoz = loader_resultadosvoz.loadHistoricalFile('Resultados') # Por precaución eliminar tres columnas posteriores a dic-18.
#resultadosvoz = pd.DataFrame() # Usar si no hay resultados
#churn = loader.loadHistoricalFile('Churn')
#calidad = loader.loadHistoricalFile('Calidad')
#qnp = loader.loadHistoricalFile('QNP') 
#tracking = loader.loadHistoricalFile('Tracking') # Asegurarse el nombre del kpi en resultados
#epa = loader.loadHistoricalFile('EPA') # Asegurarse el nombre del kpi en resultados
#resultadosvoz = resultadosvoz.append(churn, ignore_index = True)
#resultadosvoz = resultadosvoz.append(calidad, ignore_index = True)
#resultadosvoz = resultadosvoz.append(qnp, ignore_index = True)
#resultadosvoz = resultadosvoz.append(tracking, ignore_index = True)
#resultadosvoz = resultadosvoz.append(epa, ignore_index = True)
#"""

handler.show_logs()

Output(layout=Layout(border='solid', height='250px', width='100%'), outputs=({'name': 'stdout', 'output_type':…

In [5]:
# Preparando la ultima data : HC, VAS

#hc = loader.loadFile('HC')
#vas = dbmanager.loadData('View_VAS_Voz')
#vas = pd.DataFrame() # temporal mientras se arregla el código

#periodo = loader.getPeriodo() # arriba se repite


"""
criteria = {'VAS' : {'colgroupby' : ['GERENCIA2' , 'ZONAVENTA', 'DEPARTAMENTO', 'VENDEDOR_CROSS_SELLING'], 
                     'colsum' : 'ACCESS_REAL'},
            'HC' : {'colfilter' : 'CARGO', 'colfilteritem' : 'CONSULTOR', 'colgroupby' : 'DATOS', 'colsum' : 'VENDEDORES'}}

otherparams = {'periodo' : periodo.upper(), 'frames' : {'HC' : hc, 'VAS' : vas}, 'criteria': criteria}
otherdfs = loader.prepareOtherPlainFiles(otherparams) 


for df in otherdfs.keys():
    resultadosvoz = resultadosvoz.append(otherdfs[df], ignore_index = True)

"""


#"""
paramsvoz = {'tipo' : 'voz',
             'periodo' : periodo.upper(),
             'keycol': 'LOGIN',
             'logins' : logins, 
             'metricasconjuntas' : metricasconjuntas,
             'kpis' : kpisvoz, 
             'comisionantes' : comisionantesvoz, 
             'cuotas' : cuotasvoz,
             'resultados' : resultadosvoz}


panelvoz = ecomis.LoadFileProcess(section_cuotasvoz).prepareHistoricalFiles(paramsvoz)          
panelvoz.to_csv(posixpath.join(testpath, month + '_panelvoz.csv'),encoding='utf-8-sig')
handler.show_logs()
#"""

C:\Users\jsaenza\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
C:\Users\jsaenza\Anaconda3\lib\site-packages\ecomis\Lib\datapreparation.py:47: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  r1 = values_detected.ix[i].values[0]
C:\Users\jsaenza\Anaconda3\lib\site-packages\ecomis\Lib\datapreparation.py:103: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-depr

Output(layout=Layout(border='solid', height='250px', width='100%'), outputs=({'name': 'stdout', 'output_type':…

In [ ]:
cuotasvoz.to_csv(testpath + 'cuotas_brutas.csv')

In [5]:
testpath

'/Users/johnnysaenz/OneDrive - Entel Peru S.A/Comisiones/MercadoEmpresas/Data Fuente Comisiones/test'